In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
CHANNEL_NAME = os.getenv("CHANNEL_NAME")

# 슬랙 봇 토큰
SLACK_BOT_TOKEN = SLACK_BOT_TOKEN

In [10]:
from slack_sdk import WebClient

# Slack 클라이언트 초기화
client = WebClient(token=SLACK_BOT_TOKEN)  # 직접 토큰 입력 (환경 변수 대신)

def send_message(message: str, channel: str):
    """Slack 채널에 메시지 전송"""
    try:
        client.chat_postMessage(channel=CHANNEL_NAME, text=message)
        print("메시지가 전송되었습니다.")
    except Exception as e:
        print(f"오류 발생: {e}")

if __name__ == "__main__":
    send_message("안녕하세요, Slack 테스트 메시지입니다!", "#your-channel-name")


오류 발생: WebClient.chat_postMessage() takes 1 positional argument but 2 positional arguments (and 1 keyword-only argument) were given
